In [ ]:
hf_tokens = "API_KEY"
mistal_token = "API_KEY"

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
!pip install -q mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [ ]:

import os
from mistralai import Mistral
import re


client = Mistral(api_key=mistal_token)

# Agents

You can create an Agent in https://console.mistral.ai/build/agents/new, for this notebook we will use mistral-large-2407 as the model powering our agents!

Here are the instructions provided to the agents we created:
GARP (Growth at a Reasonable Price) strategy and analyze the Nifty50 stocks:

---

### **Planning Agent Prompt:**

You are a data analytical planning assistant. Given a dataset containing "balance-sheet," "cash-flow statement," "p&l statement," and "quarterly-results" of Nifty50 stocks over a 12-year period, your task is to provide specific and simple analysis plans that apply the GARP (Growth at a Reasonable Price) strategy to find the 5 best stocks to invest in for the next 5 years.

The GARP strategy combines growth and value investing by looking at companies with consistent earnings growth and a reasonable price in terms of P/E ratio. Key factors include earnings per share (EPS) growth, price-to-earnings (P/E) ratio, return on equity (ROE), debt levels, and cash flow stability.

Return output with the following format:

- **Total number of steps:**

- **Step 1:**
  Describe the overall process to clean and prepare the dataset for analysis, including handling missing values and ensuring consistency across the different financial statements.

- **Step 2:**
  Explain how to calculate the key financial metrics relevant to GARP, such as EPS growth, P/E ratio, ROE, and debt-to-equity ratio, across the 12 years.

- **Step 3:**
  Describe how to screen for stocks based on these metrics (e.g., companies with EPS growth > 10%, P/E ratio < 25, and ROE > 15%).

- **Step 4:**
  Provide instructions on how to rank the stocks based on the GARP criteria to identify the top 5 stocks with growth potential at a reasonable price.

- **Step 5:**
  Detail the plan for summarizing the analysis results in a structured format to be provided to the Summarization Agent.

---

### **Python Agent Prompt:**

You are a Python coding assistant that only outputs Python code with explanations or comments. Given the following instructions from the Planning Agent, you need to generate Python code for analyzing a dataset containing 12 years of financial data for Nifty50 stocks, including "balance-sheet," "cash-flow statement," "p&l statement," and "quarterly-results."

You will:
1. Clean and prepare the dataset by handling missing values and ensuring consistency across the data.
2. Calculate key GARP financial metrics such as EPS growth, P/E ratio, ROE, and debt-to-equity ratio.
3. Screen the stocks based on GARP criteria (EPS growth > 10%, P/E ratio < 25, ROE > 15%).
4. Rank the stocks and identify the top 5 based on these metrics.

Ensure that your code is well-documented, modular, and efficient, with comments explaining each step. Return only the Python code.

---

### **Summarization Agent Prompt:**

You are an analysis summarization assistant. You have access to a dataset containing 12 years of financial data for Nifty50 stocks ("balance-sheet," "cash-flow statement," "p&l statement," and "quarterly-results") and the results of an analysis that applied the GARP (Growth at a Reasonable Price) strategy to identify the top 5 stocks for investment.

Your task is to generate a concise analysis report summarizing:
1. The financial health and growth potential of the selected 5 stocks.
2. Key GARP criteria used (e.g., EPS growth, P/E ratio, ROE).
3. Any notable trends or observations from the 12 years of financial data.
4. The rationale behind the selection of each stock as a top pick for investment over the next 5 years.

The report should be structured as follows:
- **Introduction:** Brief overview of the dataset and GARP strategy.
- **Methodology:** Summary of the analysis process (how the key metrics were calculated and how stocks were screened).
- **Results:** Summary of the top 5 stocks, with a brief explanation of why each stock was chosen.
- **Conclusion:** Final remarks on the analysis and investment outlook based on the findings.

---



In [ ]:
python_agent_id = "ag:ce02e123:20240831:python-agent:f03c7fa0"
summarization_agent_id = "ag:ce02e123:20240831:summarization-agent:68a05ea6"
planning_agent_id = "ag:ce02e123:20240831:planning-agent:81aa3a67"

# Analysis Planning

In [ ]:
import requests
import json

def run_analysis_planning_agent(query):
    """
    Sends a user query to a Python agent and returns the response.

    Args:
        query (str): The user query to be sent to the Python agent.

    Returns:
        str: The response content from the Python agent.
    """
    print("### Run Planning agent")
    print(f"User query: {query}")
    try:
        response = client.agents.complete(
            agent_id= planning_agent_id,
            messages = [
                {
                    "role": "user",
                    "content":  query
                },
            ]
        )
        result = response.choices[0].message.content
        return result
    except Exception as e:
        print(f"Request failed: {e}. Please check your request.")
        return None

In [ ]:
query = """
Load this data: https://github.com/Pritam3355/Datasets/blob/main/fundamental.csv

The dataset consists of 21866 datapoints and has eight columns:
- category
- report_type
- stock_symbol
- year
- month
- value_type
"""

In [ ]:
planning_result = run_analysis_planning_agent(query)

### Run Planning agent
User query: 
Load this data: https://raw.githubusercontent.com/fivethirtyeight/data/master/bad-drivers/bad-drivers.csv

The dataset consists of 51 datapoints and has eight columns:
- State
- Number of drivers involved in fatal collisions per billion miles
- Percentage Of Drivers Involved In Fatal Collisions Who Were Speeding
- Percentage Of Drivers Involved In Fatal Collisions Who Were Alcohol-Impaired
- Percentage Of Drivers Involved In Fatal Collisions Who Were Not Distracted
- Percentage Of Drivers Involved In Fatal Collisions Who Had Not Been Involved In Any Previous Accidents
- Car Insurance Premiums (
)



In [ ]:
print(planning_result)

## Total number of steps: 3

## Step 1:

**Analysis Plan:** Load the dataset from the provided URL.

**Detailed Instructions:**
1. Use the `pandas` library to read the CSV file from the URL.
2. Store the data in a DataFrame.

**Suggested Python Code:**
```python
import pandas as pd

# Load the dataset
url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/bad-drivers/bad-drivers.csv"
df = pd.read_csv(url)
```

## Step 2:

**Analysis Plan:** Inspect the first few rows of the dataset to understand its structure.

**Detailed Instructions:**
1. Use the `head()` method to display the first five rows of the DataFrame.

**Suggested Python Code:**
```python
# Inspect the first few rows of the dataset
print(df.head())
```

## Step 3:

**Analysis Plan:** Summarize the dataset to get a statistical overview.

**Detailed Instructions:**
1. Use the `describe()` method to get a summary of the numerical columns in the DataFrame.
2. Use the `info()` method to get a concise summary of the 

# Generate and execute Python code for each planning step

In [ ]:
class PythonAgentWorkflow:
    def __init__(self):
        pass

    def extract_pattern(self, text, pattern):
        """
        Extracts a pattern from the given text.

        Args:
            text (str): The text to search within.
            pattern (str): The regex pattern to search for.

        Returns:
            str: The extracted pattern or None if not found.
        """
        match = re.search(pattern, text, flags=re.DOTALL)
        if match:
            return match.group(1).strip()
        return None

    def extract_step_i(self, planning_result, i, n_step):
        """
        Extracts the content of a specific step from the planning result.

        Args:
            planning_result (str): The planning result text.
            i (int): The step number to extract.
            n_step (int): The total number of steps.

        Returns:
            str: The extracted step content or None if not found.
        """
        if i < n_step:
            pattern = rf'## Step {i}:(.*?)## Step {i+1}'
        elif i == n_step:
            pattern = rf'## Step {i}:(.*)'
        else:
            print(f"Invalid step number {i}. It should be between 1 and {n_step}.")
            return None

        step_i = self.extract_pattern(planning_result, pattern)
        if not step_i:
            print(f"Failed to extract Step {i} content.")
            return None

        return step_i

    def extract_code(self, python_agent_result):
          """
          Extracts Python function and test case from the response content.

          Args:
              result (str): The response content from the Python agent.

          Returns:
              tuple: A tuple containing the extracted Python function and a retry flag.
          """
          retry = False
          # # Check if python_agent_result is None. If so, set retry to True and return None, True
          # if python_agent_result is None:
          #     retry = True
          #     return None, retry
          print("### Extracting Python code")
          python_code = self.extract_pattern(python_agent_result, r'```python(.*?)```')
          if not python_code:
              retry = True
              print("Python function failed to generate or wrong output format. Setting retry to True.")

          return python_code, retry

    def run_python_agent(self, query):
        """
        Sends a user query to a Python agent and returns the response.

        Args:
            query (str): The user query to be sent to the Python agent.

        Returns:
            str: The response content from the Python agent.
        """
        print("### Run Python agent")
        print(f"User query: {query}")
        try:
            response = client.agents.complete(
                agent_id= python_agent_id,
                messages = [
                    {
                        "role": "user",
                        "content":  query
                    },
                ]
            )
            result = response.choices[0].message.content
            return result

        except Exception as e:
            print(f"Request failed: {e}. Please check your request.")
            return None

    def check_code(self, python_function, state):
        """
        Executes the Python function and checks for any errors.

        Args:
            python_function (str): The Python function to be executed.

        Returns:
            bool: A flag indicating whether the code execution needs to be retried.

        Warning:
            This code is designed to run code that’s been generated by a model, which may not be entirely reliable.
            It's strongly recommended to run this in a sandbox environment.
        """
        retry = False
        try:
            print(f"### Python function to run: {python_function}")
            exec(python_function, state)
            print("Code executed successfully.")
        except Exception:
            print(f"Code failed.")
            retry = True
            print("Setting retry to True")
        return retry

    def process_step(self, planning_result, i, n_step, max_retries, state):
        """
        Processes a single step, including retries.

        Args:
            planning_result (str): The planning result text.
            i (int): The step number to process.
            n_step (int): The total number of steps.
            max_retries (int): The maximum number of retries.

        Returns:
            str: The extracted step content or None if not found.
        """

        retry = True
        j = 0
        while j < max_retries and retry:
            print(f"TRY # {j}")
            j += 1
            step_i = self.extract_step_i(planning_result, i, n_step)
            if step_i:
                print(step_i)
                python_agent_result = self.run_python_agent(step_i)
                python_code, retry = self.extract_code(python_agent_result)
                print(python_code)
                retry = self.check_code(python_code, state)
        return None

    def workflow(self, planning_result):
        """
        Executes the workflow for processing planning results.

        Args:
            planning_result (str): The planning result text.
        """
        state = {}
        print("### ENTER WORKFLOW")
        n_step = int(self.extract_pattern(planning_result, '## Total number of steps:\s*(\d+)'))
        for i in range(1, n_step + 1):
            print(f"STEP # {i}")
            self.process_step(planning_result, i, n_step, max_retries=2, state=state)


        print("### Exit WORKFLOW")
        return None


In [ ]:
import sys
import io

# See the output of print statements in the console while also capturing it in a variable,
class Tee(io.StringIO):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.original_stdout = sys.stdout

    def write(self, data):
        self.original_stdout.write(data)
        super().write(data)

    def flush(self):
        self.original_stdout.flush()
        super().flush()

# Create an instance of the Tee class
tee_stream = Tee()

# Redirect stdout to the Tee instance
sys.stdout = tee_stream


Python_agent = PythonAgentWorkflow()
Python_agent.workflow(planning_result)

# Restore the original stdout
sys.stdout = tee_stream.original_stdout

# Get the captured output
captured_output = tee_stream.getvalue()

### ENTER WORKFLOW
STEP # 1
TRY # 0
**Analysis Plan:** Load the dataset from the provided URL.

**Detailed Instructions:**
1. Use the `pandas` library to read the CSV file from the URL.
2. Store the data in a DataFrame.

**Suggested Python Code:**
```python
import pandas as pd

# Load the dataset
url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/bad-drivers/bad-drivers.csv"
df = pd.read_csv(url)
```
### Run Python agent
User query: **Analysis Plan:** Load the dataset from the provided URL.

**Detailed Instructions:**
1. Use the `pandas` library to read the CSV file from the URL.
2. Store the data in a DataFrame.

**Suggested Python Code:**
```python
import pandas as pd

# Load the dataset
url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/bad-drivers/bad-drivers.csv"
df = pd.read_csv(url)
```
### Extracting Python code
import pandas as pd

# Load the dataset
url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/bad-drivers/bad-drivers

# Summarization

In [ ]:
response = client.agents.complete(
    agent_id= summarization_agent_id,
    messages = [
        {
            "role": "user",
            "content":  query + captured_output
        },
    ]
)
result = response.choices[0].message.content


In [ ]:
print(result)

## Analysis Report

### Dataset Overview

The dataset comprises information on driving safety and insurance data for 51 states in the United States. It contains the following columns:

1. **State**: The name of the state.
2. **Number of drivers involved in fatal collisions per billion miles**: The rate of fatal collisions per billion miles driven.
3. **Percentage Of Drivers Involved In Fatal Collisions Who Were Speeding**: The percentage of drivers involved in fatal collisions who were speeding.
4. **Percentage Of Drivers Involved In Fatal Collisions Who Were Alcohol-Impaired**: The percentage of drivers involved in fatal collisions who were alcohol-impaired.
5. **Percentage Of Drivers Involved In Fatal Collisions Who Were Not Distracted**: The percentage of drivers involved in fatal collisions who were not distracted.
6. **Percentage Of Drivers Involved In Fatal Collisions Who Had Not Been Involved In Any Previous Accidents**: The percentage of drivers involved in fatal collisions who